## TripAdvisor Scraper
### Class Names: 

RESTAURANTS: 
 - Name: tag: a, "BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"
 - Ratings: tag: svg, "UctUV d H0"
 - Price: tag: span, "YECgr"

ATTRACTIONS:
 - Name: tag: div, "biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"
 - Ratings: tag: svg, "UctUV d H0 hzzSG"
 - Description: tag: span, "yRoCR"

In [1]:
import time
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os 
import re
chromedriver_path = '/Users/adithisatish/TUM/Academics/SoSe24/guided_research/chromedriver-mac-arm64/chromedriver'

In [2]:
csv_path = "../../europeancities-wikivoyage-tripadvisor/data-sources/tripadvisor/"

In [123]:
# base_url = "https://www.tripadvisor.com/"

# service = Service(executable_path=chromedriver_path)
# options = Options()
# # options.add_argument('--headless=new')
# # options.add_argument("--incognito")
# driver = webdriver.Chrome(options=options, service=service)

# driver.get(base_url)

In [6]:
path_to_folder = "../../europeancities-wikivoyage-tripadvisor/data-sources/tripadvisor/"

def preprocess_attractions(city, attractions_list, description_list):
    attractions_df = pd.DataFrame(columns=['city','attraction','description'])
    pattern = re.compile(r'^\d+\.\s')
    attractions_list = [s for s in attractions_list if pattern.match(s)]
    attractions_list = list(map(lambda x: x.split(". ")[1], attractions_list))
    description_list = description_list[:12]
    attractions_df['attraction'] = attractions_list
    attractions_df['description'] = description_list

    attractions_df['city'] = city
    if attractions_df.empty: 
        print("Failed to get data for city: {0}".format(city))
    else:
        attractions_df.to_csv(path_to_folder + city + "/attractions.csv", index=False)
    print("Successfully saved attractions.csv for city: {0}".format(city))

def preprocess_restaurants(city, restaurants_list, prices_list):
    df = pd.DataFrame(columns=['city','restaurant','price'])
    restaurants_list = list(filter(('See all ').__ne__, restaurants_list))
    restaurants_list = list(map(lambda x: x.split(". ")[1], restaurants_list))

    df['restaurant'] = restaurants_list
    df['price'] = prices_list
    df['city'] = city

    if df.empty: 
        print("Failed to get data for city: {0}".format(city))
    else:
        df.to_csv(path_to_folder + "restaurants/original/" + city + ".csv", index=False)
        print("Successfully saved restaurants.csv for city: {0}".format(city))

def get_data(city: str, data_type = "Things to Do"):
    # data_type can either be "Things to do" or "Restaurants"
    base_url = "https://www.tripadvisor.com/"

    options = Options()
    options.add_argument('--headless=new')
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    # driver = webdriver.Safari()
    # driver.implicitly_wait(10)
    driver.get(base_url)

    time.sleep(5)
    # click on I accept cookies
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div[2]/div[1]/div/div[2]/div/div[1]/button").click()
    # time.sleep(5)

    wait = WebDriverWait(driver, 30)
    # click on search bar
    search = wait.until(EC.element_to_be_clickable((By.XPATH,
                                                    "//input[@class='hUpcN _G G_ B- z F1 _J w Cj R0 NBfGt H3' and "
                                                    "@placeholder='Places to go, things to do, hotels...']")))
    search.send_keys(city)
    search.send_keys(Keys.RETURN)
    time.sleep(10)

    # clicking on the first result
    driver.find_element(By.XPATH,
                        "/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div["
                        "1]/div/div[2]/div/div/div/div/div/div/div[1]").click()

    time.sleep(10)
    window_after = driver.window_handles[1]
    driver.switch_to.window(window_after)
    time.sleep(5)
    # clicking on things to do
    wait.until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, data_type))).click()
    # switching windows
    next_window = driver.window_handles[1]
    driver.switch_to.window(next_window)
    time.sleep(5)
    if data_type == "Things to Do": 
        try:
            # finding attraction info table
            attractions = driver.find_elements('xpath','//div[@class="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"]')
            description = driver.find_elements('xpath','//div[@class="biGQs _P pZUbB KxBGd"]')
            # ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

            attractions_list = []
            for a in range(len(attractions)):
                attractions_list.append(attractions[a].text)

            description_list = []
            for d in range(len(description)):
                description_list.append(description[d].text)

        except Exception as e:
            print("Error - could not get attractions and description for city: {0}, {1}".format(city, e))
    elif data_type == "Restaurants":
        try:
            # finding attraction info table
            restaurants = driver.find_elements('xpath','//a[@class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"]')
            prices = driver.find_elements('xpath','//span[@class="YECgr"]')
            # ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

            restaurants_list = []
            for r in range(len(restaurants)):
                restaurants_list.append(restaurants[r].text)

            prices_list = []
            for p in range(len(prices)):
                prices_list.append(prices[p].text)

        except Exception as e:
            print("Error - could not get attractions and description for city: {0}, {1}".format(city, e))

    driver.quit()

    if data_type == "Things to Do":
        preprocess_attractions(city, attractions_list, description_list)
    else:
        preprocess_restaurants(city, restaurants_list, prices_list)


In [7]:
def get_attractions(city, url):
    driver = webdriver.Safari()
    driver.get(url)
    time.sleep(2)

    attractions = driver.find_elements('xpath','//div[@class="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"]')
    description = driver.find_elements('xpath','//div[@class="biGQs _P pZUbB KxBGd"]')
    # ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

    attractions_list = []
    for a in range(len(attractions)):
        attractions_list.append(attractions[a].text)

    description_list = []
    for d in range(len(description)):
        description_list.append(description[d].text)

    driver.close()

    preprocess_attractions(city, attractions_list, description_list)

In [8]:
def get_restaurants(city, url): 
    
    # PROXY = 'https://217.64.149.146:8118'
    # chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--proxy-server=%s' % PROXY)
    # chrome_options.add_argument("--enable-javascript")
    # driver = webdriver.Chrome(options=chrome_options)
    # options.add_argument("--incognito")
    # driver = webdriver.Chrome(options=options)
    driver = webdriver.Safari()
    # time.sleep(2)
    driver.get(url)
    time.sleep(3)

    restaurants = driver.find_elements('xpath','//a[@class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"]')
    prices = driver.find_elements('xpath','//span[@class="YECgr"]')
    # ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

    restaurants_list = []
    for r in range(len(restaurants)):
        restaurants_list.append(restaurants[r].text)

    prices_list = []
    for p in range(len(prices)):
        prices_list.append(prices[p].text)
        
    driver.close()

    preprocess_restaurants(city, restaurants_list, prices_list)

In [9]:
cities = pd.read_csv("../europeancities.csv")
city_mapping = pd.read_csv(path_to_folder + "city_tripadvisor_mapping.csv")
city_mapping.head()

,city,attraction_url,restaurant_url
0,Amsterdam,https://www.tripadvisor.com/Attractions-g18859...,https://www.tripadvisor.com/Restaurants-g18859...
1,Barcelona,https://www.tripadvisor.com/Attractions-g18749...,https://www.tripadvisor.com/Restaurants-g18749...
2,Berlin,https://www.tripadvisor.com/Attractions-g18732...,https://www.tripadvisor.com/Restaurants-g18732...
3,Bologna,https://www.tripadvisor.com/Attractions-g18780...,https://www.tripadvisor.com/Restaurants-g18780...
4,Bordeaux,https://www.tripadvisor.com/Attractions-g18707...,https://www.tripadvisor.com/Restaurants-g18707...


In [10]:
attractions = 0

if attractions:
    for index, row in city_mapping.iterrows():
        # print(row)
        get_attractions(row['city'], row['attraction_url'])

In [14]:
for index, row in city_mapping.iterrows():
    # print(row)
    get_restaurants(row['city'], row['restaurant_url'])
    # get_data(row['city'], 'Restaurants')

KeyboardInterrupt: 

### URLs

In [35]:
restaurant_url = "https://www.tripadvisor.com/Restaurants-g187849-Milan_Lombardy.html"
attraction_url = "https://www.tripadvisor.com/Attractions-g187849-Activities-Milan_Lombardy.html"
# hotel_url

city = 'Milan'

### RESTAURANTS

In [67]:
df = pd.DataFrame(columns=['city','restaurant','price']) # creates master dataframe 

# PROXY = 'https://20.205.61.143:8123'
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % PROXY)
# chrome_options.add_argument("--enable-javascript")
# driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()
options = Options()
options.add_argument('--headless=new')
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

driver.get(restaurant_url)
time.sleep(2)

restaurants = driver.find_elements('xpath','//a[@class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"]')
prices = driver.find_elements('xpath','//span[@class="YECgr"]')
# ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

restaurants_list = []
for r in range(len(restaurants)):
    restaurants_list.append(restaurants[r].text)

prices_list = []
for p in range(len(prices)):
    prices_list.append(prices[p].text)
    
driver.close()

restaurants_list = list(filter(('See all ').__ne__, restaurants_list))
restaurants_list = list(map(lambda x: x.split(". ")[1], restaurants_list))

df['restaurant'] = restaurants_list
df['price'] = prices_list
df['city'] = city
path = csv_path + city
df.to_csv(path + "/restaurants.csv", index=False)

In [34]:
restaurants_list

[]

### ATTRACTIONS

In [51]:
attractions_df = pd.DataFrame(columns=['city','attraction','description'])

driver = webdriver.Safari()
driver.get(attraction_url)
time.sleep(2)

attractions = driver.find_elements('xpath','//div[@class="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"]')
description = driver.find_elements('xpath','//div[@class="biGQs _P pZUbB KxBGd"]')
# ratings = driver.find_element('xpath', '//svg[@class="UctUV d H0"]')

attractions_list = []
for a in range(len(attractions)):
    attractions_list.append(attractions[a].text)

description_list = []
for d in range(len(description)):
    description_list.append(description[d].text)

driver.close()

pattern = re.compile(r'^\d+\.\s')
attractions_list = [s for s in attractions_list if pattern.match(s)]
attractions_list = list(map(lambda x: x.split(". ")[1], attractions_list))
description_list = description_list[:12]
attractions_df['attraction'] = attractions_list
attractions_df['description'] = description_list
attractions_df['city'] = city
path_to_folder = "../europeancities-wikivoyage-tripadvisor/data-sources/tripadvisor/"

attractions_df.to_csv(path_to_folder + city + "/attractions.csv")
print("Successfully saved attractions.csv for city: {0}".format(city))